In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay


In [6]:
# Definir la ruta a los directorios de datos
train_dir = 'D:/descargas/Proyecto_L (2)/Proyecto_L/dataset_final/train'
val_dir = 'D:/descargas/Proyecto_L (2)/Proyecto_L/dataset_final/val'
test_dir = 'D:/descargas/Proyecto_L (2)/Proyecto_L/dataset_final/test'

# Definir el tamaño de las imágenes
img_size = (224, 224)

# Preprocesamiento de las imágenes de entrada
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest',
                                   brightness_range=[0.7, 1.3])

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical')


Found 41907 images belonging to 6 classes.
Found 11977 images belonging to 6 classes.
Found 5990 images belonging to 6 classes.


In [7]:
# 1. Cargar MobileNetV2 pre-entrenado (sin la capa superior)
base_model = MobileNetV2(
    weights='imagenet',  # Pesos pre-entrenados en ImageNet
    include_top=False,    # No incluir la capa densa final
    input_shape=(224, 224, 3)
)

# 2. Congelar las capas base (opcional, para fine-tuning)
base_model.trainable = True
# Descongelar las últimas 50 capas
for layer in base_model.layers[-50:]:
    layer.trainable = True

# 3. Añadir nuevas capas para tu problema
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)  # Normalización para mejorar la estabilidad
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(6, activation='softmax')(x)

# 4. Crear modelo final
model = Model(inputs=base_model.input, outputs=predictions)

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = Adam(learning_rate=lr_schedule)
# 5. Compilar el modelo
model.compile(
    optimizer = optimizer,  # Tasa de aprendizaje más baja
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,982,854 (11.38 MB)

 Trainable params: 2,947,462 (11.24 MB)

 Non-trainable params: 35,392 (138.25 KB)

In [8]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np

y_train = train_generator.classes

# Callbacks

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

checkpoint = ModelCheckpoint(
    filepath='modelo_epoca_{epoch:02d}_val_loss_{val_loss:.4f}.h5',  # Ahora en formato .h5
    monitor='val_loss',
    save_best_only=False,  # Guarda cada época, no solo la mejor
    save_weights_only=False,
    verbose=1
)

# Cálculo de pesos de clase
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[early_stopping, checkpoint]
)



d:\descargas\Proyecto_L (2)\Proyecto_L\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3711 - loss: 1.6925
Epoch 1: saving model to modelo_epoca_01_val_loss_1.0522.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 2089s 2s/step - accuracy: 0.3712 - loss: 1.6923 - val_accuracy: 0.6019 - val_loss: 1.0522
Epoch 2/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5970 - loss: 1.0383
Epoch 2: saving model to modelo_epoca_02_val_loss_0.8396.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1816s 1s/step - accuracy: 0.5970 - loss: 1.0382 - val_accuracy: 0.6917 - val_loss: 0.8396
Epoch 3/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6699 - loss: 0.8624
Epoch 3: saving model to modelo_epoca_03_val_loss_0.7332.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1762s 1s/step - accuracy: 0.6699 - loss: 0.8624 - val_accuracy: 0.7296 - val_loss: 0.7332
Epoch 4/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7105 - loss: 0.7619
Epoch 4: saving model to modelo_epoca_04_val_loss_0.7531.h5


...

1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1745s 1s/step - accuracy: 0.8232 - loss: 0.4855 - val_accuracy: 0.7954 - val_loss: 0.5862
Epoch 12/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8297 - loss: 0.4546
Epoch 12: saving model to modelo_epoca_12_val_loss_0.5682.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1744s 1s/step - accuracy: 0.8297 - loss: 0.4546 - val_accuracy: 0.7950 - val_loss: 0.5682
Epoch 13/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8369 - loss: 0.4402
Epoch 13: saving model to modelo_epoca_13_val_loss_0.5367.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1745s 1s/step - accuracy: 0.8369 - loss: 0.4402 - val_accuracy: 0.8072 - val_loss: 0.5367
Epoch 14/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8460 - loss: 0.4159
Epoch 14: saving model to modelo_epoca_14_val_loss_0.5258.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1736s 1s/step - accuracy: 0.8460 - loss: 0.4159 - val_accuracy: 0.8113 - val_loss: 0.5258
Epoch 15/15
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8520 - loss: 0.4008
Epoch 15: saving model to modelo_epoca_15_val_loss_0.5118.h5


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 1742s 1s/step - accuracy: 0.8520 - loss: 0.4008 - val_accuracy: 0.8192 - val_loss: 0.5118


In [9]:
model.save('modelo_nuevo_L.h5')